<img src="data/photutils_banner.svg" width=500 alt="Photutils logo" style="margin-left: 0;">

# Photutils

- Code: https://github.com/astropy/photutils
- Documentation: https://photutils.readthedocs.io/en/stable/
- Issue Tracker:  https://github.com/astropy/photutils/issues


<div class="alert alert-block alert-info">
<h2 style="margin-top: 0">In this notebook, we will cover:</h2>

- Local background estimation
- Aperture mask objects
</div>

---

# Local Background Subtraction

## Preliminaries

In [ ]:
# initial imports
import numpy as np
import matplotlib.pyplot as plt

# change some default plotting parameters
import matplotlib as mpl
mpl.rcParams['image.origin'] = 'lower'
mpl.rcParams['image.interpolation'] = 'nearest'

# Run the %matplotlib magic command to enable inline plotting
# in the current notebook.  Choose one of these:
%matplotlib inline
# %matplotlib notebook

### Load the data

We'll start by reading science data and error arrays from FITS files located in the [**data/**](data) subdirectory.  The FITS files contain 2D cutout images from the [Hubble Extreme-Deep Field (XDF)](https://archive.stsci.edu/prepds/xdf/) taken with the [Wide Field Camera 3 (WFC3)](https://www.stsci.edu/hst/instrumentation/wfc3) IR channel in the F160W filter (centered at ~1.6 $\mu m$).

In [ ]:
from astropy.io import fits
from astropy.wcs import WCS

sci_fn = 'data/xdf_hst_wfc3ir_60mas_f160w_sci.fits'
rms_fn = 'data/xdf_hst_wfc3ir_60mas_f160w_rms.fits'

hdr = fits.getheader(sci_fn)
data = fits.getdata(sci_fn)
error = fits.getdata(rms_fn)
wcs = WCS(hdr)

In [ ]:
# calculate the total error:  background plus source Poisson error
from photutils.utils import calc_total_error

eff_gain = hdr['TEXPTIME']
total_error = calc_total_error(data, error, eff_gain)

The background in the XDF image has already been subtracted.  Let's add a background of 5 e-/s so we'll have something to subtract.

In [ ]:
data += 5.

In [ ]:
# display the data
from astropy.visualization import simple_norm

plt.figure(figsize=(8, 8))
norm = simple_norm(data, 'sqrt', percent=99.5)
plt.imshow(data, norm=norm)
plt.colorbar(shrink=0.8)

## Perform aperture photometry at multiple positions

In [ ]:
# define three apertures
from photutils.aperture import CircularAperture

positions = [(90.73, 59.43), (73.63, 139.41), (43.62, 61.63)]
radius = 5.  # pixels
apertures = CircularAperture(positions, r=radius)

In [ ]:
# plot the apertures
plt.figure(figsize=(8, 8))
plt.imshow(data, norm=norm)
apertures.plot(color='red', lw=2);

In [ ]:
# perform aperture photometry for the three sources
# data here includes background, so the aperture sums are *not* the source fluxes
# ideally the data array should be background subtracted before running aperture_photometry
import astropy.units as u
from photutils.aperture import aperture_photometry

unit = u.electron / u.s
phot = aperture_photometry(data << unit, apertures, error=total_error << unit)
phot

## Local background estimation

Let's start by creating circular-annulus apertures at the same positions.

Here we're define each circular annulus to have an inner and outer radius of 10 and 15 pixels, respectively.

The circular-annulus apertures will be used for local background estimation around the sources.

In [ ]:
from photutils.aperture import CircularAnnulus

positions = [(90.73, 59.43), (73.63, 139.41), (43.62, 61.63)]
aper = CircularAperture(positions, r=5)
bkg_aper = CircularAnnulus(positions, r_in=10., r_out=15.)
apers = [aper, bkg_aper]

In [ ]:
# plot the apertures
plt.figure(figsize=(8, 8))
plt.imshow(data, norm=norm)
aper_patches = aper.plot(color='white', lw=2, label='Photometry aperture')
ann_patches = bkg_aper.plot(color='orange', lw=2, label='Background annulus', hatch='///')
handles = (aper_patches[0], ann_patches[0])
plt.legend(loc=(0.03, 0.90), facecolor='#458989', labelcolor='white',
              handles=handles, prop={'weight': 'bold', 'size': 11});

### Simple local background estimation

This first example uses the mean value in circular annulus as the background value.  We'll use the `aperture_photometry` function to calculate the pixel sum in the circular annulus, from which we can calculate the mean background value.

In [ ]:
phot = aperture_photometry(data, apers)
phot.rename_column('aperture_sum_0', 'aperture_sum')
phot.rename_column('aperture_sum_1', 'annulus_sum')
phot

Note that the fluxes cannot be simply subtracted because the aperture areas are different.

First, calculate the mean background level (per pixel) in the annuli.

In [ ]:
phot['annulus_mean'] = phot['annulus_sum'] / bkg_aper.area
phot

Then multiply it by the circular aperture area to get the total background in the apertures.

In [ ]:
phot['aperture_bkg'] = phot['annulus_mean'] * aper.area
phot

Now subtract the background.

In [ ]:
# subtract the background
flux_bkgsub = phot['aperture_sum'] - phot['aperture_bkg']

phot['aperture_sum_bkgsub'] = flux_bkgsub
phot

### Custom local background subtraction

One can use aperture masks to directly access the pixel values in an aperture.  This allows for advanced local background subtraction including, for example, sigma clipping.

Aperture masks are created using the `to_mask()` method.

In [ ]:
bkg_mask = bkg_aper.to_mask()

`bkg_mask` is a list of [ApertureMask](https://photutils.readthedocs.io/en/latest/api/photutils.aperture.ApertureMask.html#photutils.aperture.ApertureMask) objects, one for each aperture position.

In [ ]:
bkg_mask

Let's plot the first aperture mask.

In [ ]:
plt.figure(figsize=(6, 6))
plt.imshow(bkg_mask[0])  # index 0 means first mask
plt.colorbar(shrink=0.8);

The mask pixel values are between 0 and 1 (inclusive), indicating the fractional overlap of the aperture with the pixel grid.  The fractional values (between 0 and 1) on the edges are because the default overlap method is `'exact'`.  We can use other methods, e.g., `'center'`, where pixels are either completely in or out of the aperture depending on whether the pixel center is in or out of the aperture.  In that case, the mask pixel values will be either 0 or 1 (no fractional values).

In [ ]:
plt.figure(figsize=(6, 6))
bkg_mask = bkg_aper.to_mask(method='center')
plt.imshow(bkg_mask[0])
plt.colorbar(shrink=0.8);

The pixel values in the above mask are either 0 or 1.  **This type of mask (i.e., method='center') is strongly recommended for local background estimation.**

One could use the `'exact'` mask, but it requires using statistical functions that can handle partial-pixel weights.  That introduces a lot of unnecessary complexity when the aperture is simply being used to estimate the local background -- whole pixels are fine, assuming you have a sufficient number of them.

We can now use the `ApertureMask` `get_values()` method to get the mask-weighted pixel values from the data as a 1D array.  Because the mask contains only 0 or 1 values, the result is simply the data pixel values located within the annulus aperture.

In [ ]:
plt.figure(figsize=(6, 6))
bmask = bkg_mask[0]  # first aperture mask
aper_data = bmask.get_values(data)
plt.plot(aper_data)
plt.axhline(5.0, color='orange');

In [ ]:
# distribution of data values in the annulus aperture
plt.hist(aper_data, bins=20);
plt.axvline(5.0, color='orange');

You can then use your favorite statistical estimator on this data to estimate the background level.

In [ ]:
# simple mean and median
np.mean(aper_data), np.median(aper_data)

In [ ]:
# sigma-clipped mean and median
from astropy.stats import sigma_clipped_stats

mean_sigclip, median_sigclip, std_sigclip = sigma_clipped_stats(aper_data)
mean_sigclip, median_sigclip

In [ ]:
# biweight "mean"
from astropy.stats import biweight_location

biweight_location(aper_data)

These background estimates represent the background *per pixel* for the first source only.  Like the first  local background example, be sure to calculate the total background within the circular aperture before subtracting.  The area of the `'exact'` circular aperture that we used for photometry is returned by its `area()` method.

In [ ]:
# total background in circular aperture of the first source
# estimated using a sigma-clipped median in the circular annulus
bkg_total = aper.area * median_sigclip
bkg_total

Now let's subtract the background from the first source.

In [ ]:
phot['aperture_sum'][0] - bkg_total

Because there were not any outliers in the background annulus, the result is very close to our first local background estimate (based on a simple mean).

In [ ]:
phot['aperture_sum_bkgsub'][0]

## Putting it all together

Above was a very pedagogical description of the underlying methods for local background subtraction for a single source.

However, it's quite straightforward to do this for all of the sources in just a few lines of code.  For this example, we'll again use the sigma-clipped median of the pixels in the background annuli for the background estimates of each source.

First, let's calculate the sigma-clipped median values in each of the background annuli.

In [ ]:
bkg_median = []
bkg_mask = bkg_aper.to_mask(method='center')
for mask in bkg_mask:
    aper_data = bmask.get_values(data)
    
    # perform a sigma-clipped median
    _, median_sigclip, _ = sigma_clipped_stats(aper_data)
    bkg_median.append(median_sigclip)
    
bkg_median = np.array(bkg_median)
bkg_median

Again, the above values are the background *per pixel*.  Let's correct for the circular aperture area, subtract the background, and add them as table columns (`annulus_median`, `aperture_bkg2`, and `aperture_sum_bksub2`).

The simple mean columns are in the table for comparison.

In [ ]:
phot['annulus_median'] = bkg_median
phot['aperture_bkg2'] = bkg_median * aper.area
phot['aperture_sum_bkgsub2'] = phot['aperture_sum'] - phot['aperture_bkg2']
phot

## A few more things about `ApertureMask`

Cutouts of each source using the minimal bounding box of each aperture can be obtained using its `cutout()` method.

In [ ]:
plt.figure(figsize=(5, 5))
# bmask was defined above as the aperture mask of the first source
plt.imshow(bmask.cutout(data))

The `multiply()` method produces a 2D image of the aperture mask multiplied by the data cutout.

In [ ]:
plt.figure(figsize=(15, 10))

plt.subplot(1, 3, 1)
plt.imshow(bmask)
plt.title('Aperture Mask')

plt.subplot(1, 3, 2)
plt.title('Data Cutout')
plt.imshow(bmask.cutout(data))

plt.subplot(1, 3, 3)
plt.title('Aperture Mask * Data Cutout')
plt.imshow(bmask.multiply(data), vmin=4.998, vmax=5.002);

One can also plot the location of the mask on the original data using the `to_image()` method.

In [ ]:
plt.figure(figsize=(10, 10))

plt.subplot(1, 2, 1)
plt.title('Data')
plt.imshow(data, norm=norm)

plt.subplot(1, 2, 2)
plt.title('Background Annulus Mask')
plt.imshow(bmask.to_image(data.shape))

We can also use the `bbox` attribute to plot the extent of the annulus mask in the image.

In [ ]:
plt.figure(figsize=(8, 8))
plt.imshow(data, norm=norm)
bkg_aper.plot(color='orange', lw=2)
bmask.bbox.plot(color='white', lw=2)